In [2]:
import pandas as pd
import numpy as np
from covid import utils

paths = utils.create_named_tuple('filepaths.txt')
config = utils.create_named_tuple(paths.config)
labels = utils.generate_labels_from_bins(config.age_bins)
bins = config.age_bins
population = pd.read_csv('data/temp_pop.csv')

In [2]:
population

,region_id,region,0-6,7-15,16-20,21-40,41-66,67+,population
0,301,OSLO,55837,64036,33296,257985,207877,77749,696780
1,1101,EIGERSUND,1132,1719,955,3642,4898,2439,14785
2,1103,STAVANGER,11735,16218,8511,41917,47283,18451,144115
3,1106,HAUGESUND,2636,3996,2314,9836,12823,5707,37312
4,1108,SANDNES,7589,10110,5042,22845,25653,9206,80445
...,...,...,...,...,...,...,...,...,...
351,5440,BERLEVÅG,47,61,46,219,341,214,928
352,5441,TANA,166,232,154,624,1052,600,2828
353,5442,NESSEBY,33,86,40,197,295,229,880
354,5443,BÅTSFJORD,150,197,124,622,761,346,2200


In [16]:
df = pd.read_csv('data/contact_data.csv')
df.contact_age_0 = pd.cut(df['contact_age_0'], bins=bins+[110], labels=labels, include_lowest=True)
df.contact_age_1 = pd.cut(df['contact_age_1'], bins=bins+[110], labels=labels, include_lowest=True)
df_mat = pd.DataFrame(df[df.columns[:-2]].groupby(['contact_age_0', 'contact_age_1']).mean()).reset_index()
df_mat['total'] = (df_mat.home + df_mat.work + df_mat.school + df_mat.transport + df_mat.leisure)/5
N = population[population.columns[2:-1]].sum().to_numpy()
matrices = []
# for col in ['home', 'work', 'school', 'transport', 'leisure']:
#     matrix = pd.pivot_table(df_mat, values=col, index='contact_age_0', columns='contact_age_1').to_numpy()
#     symmetric_matrix = np.zeros((matrix.shape))
#     for i in range(len(N)):
#         for j in range(len(N)):
#             symmetric_matrix[i][j] = 1/(N[i]+N[j]) * (matrix[i][j] * N[i] + matrix[j][i] * N[j])
#     matrices.append(symmetric_matrix)

In [17]:
df_mat

,contact_age_0,contact_age_1,home,work,school,transport,leisure,total
0,0-6,0-6,0.177576,0.000626,0.618854,0.045099,0.247416,0.217914
1,0-6,7-15,0.498718,0.001282,0.189744,0.079487,0.348718,0.223590
2,0-6,16-20,0.366667,0.206667,0.093333,0.026667,0.353333,0.209333
3,0-6,21-40,0.574581,0.146880,0.111872,0.078387,0.255708,0.233486
4,0-6,41-66,0.430010,0.131923,0.277946,0.040282,0.228600,0.221752
5,0-6,67+,0.705128,0.012821,0.000000,0.012821,0.346154,0.215385
6,7-15,0-6,0.451693,0.001652,0.216350,0.067713,0.374897,0.222461
7,7-15,7-15,0.108252,0.002700,0.686129,0.066909,0.247638,0.222325
8,7-15,16-20,0.320812,0.021320,0.298477,0.094416,0.363452,0.219695
9,7-15,21-40,0.468691,0.194813,0.149905,0.059456,0.263757,0.227324


In [18]:
matrix = pd.pivot_table(df_mat, values='total', index='contact_age_0', columns='contact_age_1')
print(matrix)

contact_age_1       0-6      7-15     16-20     21-40     41-66       67+
contact_age_0                                                            
0-6            0.217914  0.223590  0.209333  0.233486  0.221752  0.215385
7-15           0.222461  0.222325  0.219695  0.227324  0.230990  0.215493
16-20          0.216529  0.219214  0.220469  0.215351  0.214780  0.207619
21-40          0.226534  0.215944  0.207803  0.211901  0.207648  0.209217
41-66          0.217635  0.215263  0.211931  0.207852  0.209490  0.207939
67+            0.212394  0.208264  0.207910  0.205195  0.207727  0.209524
